In [31]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


CTRL + F10

In [32]:
import pandas as pd
import numpy as np
from sklearn.feature_selection import VarianceThreshold
from sklearn.impute import KNNImputer
from sklearn.feature_selection import SelectKBest, f_regression, chi2, SelectPercentile, mutual_info_regression
from sklearn.ensemble import IsolationForest
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [33]:
folder = "drive/MyDrive/AML_Pr_1/"
raw = folder + "raw_data/"

In [34]:
raw_dataset_x = pd.read_csv(raw + 'X_train.csv',index_col=['id'])
#X = raw_dataset_x.copy()
raw_dataset_x.head()

,x0,x1,x2,x3,x4,x5,x6,x7,x8,x9,...,x822,x823,x824,x825,x826,x827,x828,x829,x830,x831
id,,,,,,,,,,,,,,,,,,,,,
0.0,14168.823171,10514.380717,3316.149698,94230.695124,102.386606,92.677127,11108.748199,10866.505510,10837.622093,10.227734,...,NaN,12352.094085,846.014651,105.132144,102.112809,2090.004260,2.691845,1234.374109,1000.784475,9285.751272
1.0,17757.037554,NaN,4101.016273,92959.527633,NaN,99.855168,10013.959449,10826.607494,10076.101597,11.436970,...,NaN,16198.071494,776.084467,106.385590,103.472030,2474.051881,2.287976,NaN,1012.626705,11750.284764
2.0,14226.656663,11029.642499,NaN,124055.600561,100.542483,92.860892,NaN,10492.342868,NaN,10.810076,...,10329.704431,13976.063780,737.040332,103.671234,109.458246,2656.083281,2.843706,888.353607,1048.810385,9553.922728
3.0,8766.012436,7384.202998,2147.308418,100157.719990,104.855061,101.929026,10050.049932,10499.521099,10525.030989,10.092109,...,10008.251395,6212.127347,329.044233,105.084488,104.858546,1097.785204,2.732257,927.752967,1048.357330,NaN
4.0,13801.016418,13269.493652,3408.316953,92048.527786,103.759758,95.789235,9667.353978,10750.783106,10618.800750,12.006773,...,10095.782015,13772.061493,NaN,NaN,100.369834,2693.053231,2.702908,1471.354073,1071.284484,9423.533063


In [35]:
raw_dataset_y = pd.read_csv(raw + 'y_train.csv',index_col=['id'])
#y = raw_dataset_y.copy()
raw_dataset_y.head()

,y
id,
0.0,74.0
1.0,51.0
2.0,70.0
3.0,52.0
4.0,85.0


In [36]:
just_scoring = False
if just_scoring:
  preprocessed = folder + "preprocessed_data/"
  x_train, x_test, y_train, y_test = train_test_split(raw_dataset_x, raw_dataset_y, test_size=0.2, random_state = 100)
else:
  preprocessed = folder + "ready_data/"
  x_train = pd.read_csv(raw + 'X_train.csv',index_col=['id'])
  x_test = pd.read_csv(raw + 'X_test.csv',index_col=['id'])
  y_train = pd.read_csv(raw + 'y_train.csv',index_col=['id'])
  y_test = pd.DataFrame({'A' : []})

In [37]:
missing_values = x_train.isna().sum()
missing_values

x0       91
x1       84
x2       88
x3       91
x4       74
       ... 
x827    101
x828     95
x829     92
x830     80
x831     96
Length: 832, dtype: int64

In [38]:
for col in x_train.columns[x_train.nunique(dropna = False) <= 100]:
    print(col, " ", x_train[col].unique())

x104   [ 0. nan]
x129   [ 0. nan]
x489   [ 0. nan]
x530   [ 0. nan]


In [39]:
x_test = x_test.drop(columns = x_train.columns[x_train.nunique(dropna = False) <= 100])
x_train = x_train.drop(columns = x_train.columns[x_train.nunique(dropna = False) <= 100])

In [40]:
x_train.isnull().sum(1).sort_values(ascending=False)

id
867.0     89
430.0     87
794.0     86
565.0     85
900.0     85
          ..
627.0     43
173.0     43
1038.0    40
268.0     39
582.0     37
Length: 1212, dtype: int64

In [41]:
nans_train = x_train[x_train.columns].isna().astype(int)
nans_train = nans_train.add_prefix("isnan_")
nans_test = x_test[x_test.columns].isna().astype(int)
nans_test = nans_test.add_prefix("isnan_")

In [42]:
#X = X.fillna(X.mean())
imputer = KNNImputer(n_neighbors=3)
#x_train = x_train.fillna(x_train.median())
#x_test = x_test.fillna(x_test.median())
x_train = pd.DataFrame(imputer.fit_transform(x_train, y_train), columns = x_train.columns, index = x_train.index)
x_test = pd.DataFrame(imputer.transform(x_test), columns = x_test.columns, index = x_test.index)

In [43]:
x_train = pd.concat([x_train, nans_train], axis=1)
x_test = pd.concat([x_test, nans_test], axis=1)
x_test
x_train

,x0,x1,x2,x3,x4,x5,x6,x7,x8,x9,...,isnan_x822,isnan_x823,isnan_x824,isnan_x825,isnan_x826,isnan_x827,isnan_x828,isnan_x829,isnan_x830,isnan_x831
id,,,,,,,,,,,,,,,,,,,,,
0.0,14168.823171,10514.380717,3316.149698,94230.695124,102.386606,92.677127,11108.748199,10866.505510,10837.622093,10.227734,...,1,0,0,0,0,0,0,0,0,0
1.0,17757.037554,10232.310907,4101.016273,92959.527633,105.293397,99.855168,10013.959449,10826.607494,10076.101597,11.436970,...,1,0,0,0,0,0,0,1,0,0
2.0,14226.656663,11029.642499,3143.476933,124055.600561,100.542483,92.860892,10435.300970,10492.342868,10346.971352,10.810076,...,0,0,0,0,0,0,0,0,0,0
3.0,8766.012436,7384.202998,2147.308418,100157.719990,104.855061,101.929026,10050.049932,10499.521099,10525.030989,10.092109,...,0,0,0,0,0,0,0,0,0,1
4.0,13801.016418,13269.493652,3408.316953,92048.527786,103.759758,95.789235,9667.353978,10750.783106,10618.800750,12.006773,...,0,0,1,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1207.0,18707.457475,13610.725702,3785.886941,113497.632841,109.526764,97.812339,11274.011935,10803.953566,10949.811419,11.024856,...,0,0,0,0,0,0,0,0,1,0
1208.0,17108.239122,12168.536128,3442.619145,98218.773311,100.468476,109.994258,11031.326117,10231.743317,10687.321177,10.238957,...,0,0,0,0,0,0,0,0,0,0
1209.0,14264.707321,9273.405761,3580.894003,101668.927699,102.620705,104.470375,9159.594864,10661.827392,10623.176915,10.132906,...,0,0,0,0,0,0,1,0,0,0


In [44]:
x_train.describe().transpose().sort_values('min')

,count,mean,std,min,25%,50%,75%,max
x173,1212.0,-1.852303e+22,2.496973e+22,-2.937933e+23,-2.871468e+22,-1.414654e+22,-3.972937e+21,7.002915e+22
x665,1212.0,4.526528e+22,3.966675e+22,-6.392632e+22,2.704526e+22,3.921405e+22,5.853573e+22,9.935455e+23
x596,1212.0,-1.835249e+14,1.234405e+14,-8.756492e+14,-2.335186e+14,-1.486501e+14,-1.082350e+14,2.926746e+14
x132,1212.0,-1.386441e+06,2.086794e+05,-2.819624e+06,-1.498551e+06,-1.367610e+06,-1.278865e+06,-5.738037e+05
x374,1212.0,-5.416921e+05,7.255622e+04,-1.002561e+06,-5.818485e+05,-5.352921e+05,-5.053880e+05,-1.607672e+05
...,...,...,...,...,...,...,...,...
x467,1212.0,1.052099e+06,2.765164e+04,1.000249e+06,1.029032e+06,1.053533e+06,1.073040e+06,1.099984e+06
x421,1212.0,1.049927e+06,2.740026e+04,1.000304e+06,1.027643e+06,1.049873e+06,1.072518e+06,1.099971e+06
x637,1212.0,1.049070e+06,2.763131e+04,1.000329e+06,1.026365e+06,1.047396e+06,1.072268e+06,1.099951e+06
x331,1212.0,1.049137e+06,2.841760e+04,1.000372e+06,1.024670e+06,1.049827e+06,1.070840e+06,1.099887e+06


In [45]:
s = StandardScaler()
x_train[[str(col) for col in x_train.columns]] = s.fit_transform(x_train)
x_test[[str(col) for col in x_train.columns]] = s.transform(x_test)

In [46]:
model=IsolationForest(n_estimators=150,max_samples='auto',contamination=float(0.2),random_state=np.random.RandomState(42), n_jobs=-1)
model.fit(np.array(x_train))
is_anomaly = model.predict(np.array(x_train))
x_train['anomaly'] = is_anomaly
y_train['anomaly'] = is_anomaly

In [47]:
x_train['anomaly'].value_counts()

 1    969
-1    243
Name: anomaly, dtype: int64

In [48]:
y_train['anomaly'].value_counts()

 1    969
-1    243
Name: anomaly, dtype: int64

In [49]:
x_train = x_train[x_train.anomaly > 0]
y_train = y_train[y_train.anomaly > 0]

In [50]:
x_train = x_train.drop('anomaly',axis=1)
y_train = y_train.drop('anomaly',axis=1)

In [51]:
x_train

,x0,x1,x2,x3,x4,x5,x6,x7,x8,x9,...,isnan_x822,isnan_x823,isnan_x824,isnan_x825,isnan_x826,isnan_x827,isnan_x828,isnan_x829,isnan_x830,isnan_x831
id,,,,,,,,,,,,,,,,,,,,,
0.0,-0.439786,-0.259312,-0.226678,-0.583575,-0.971490,-0.788129,1.130952,1.293719,1.249786,0.165677,...,3.530817,-0.272888,-0.291632,-0.293294,-0.315941,-0.301511,-0.291632,-0.286606,-0.265841,-0.293294
1.0,1.160629,-0.444346,1.587980,-0.718887,0.084219,-0.013704,-0.013749,1.147812,-1.427827,1.379006,...,3.530817,-0.272888,-0.291632,-0.293294,-0.315941,-0.301511,-0.291632,3.489114,-0.265841,-0.293294
2.0,-0.413991,0.078692,-0.625908,2.591201,-1.641252,-0.768303,0.426802,-0.074594,-0.475411,0.749991,...,-0.283221,-0.272888,-0.291632,-0.293294,-0.315941,-0.301511,-0.291632,-0.286606,-0.265841,-0.293294
3.0,-2.849547,-2.312666,-2.929110,0.047340,-0.074979,0.210041,0.023987,-0.048343,0.150672,0.029593,...,-0.283221,-0.272888,-0.291632,-0.293294,-0.315941,-0.301511,-0.291632,-0.286606,-0.265841,3.409545
4.0,-0.603835,1.548004,-0.013582,-0.815860,-0.472779,-0.452370,-0.376156,0.870522,0.480380,1.950737,...,-0.283221,-0.272888,3.428979,3.409545,-0.315941,-0.301511,-0.291632,-0.286606,-0.265841,-0.293294
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1207.0,1.584535,1.771847,0.859382,1.467335,1.621723,-0.234101,1.303750,1.064966,1.644260,0.965497,...,-0.283221,-0.272888,-0.291632,-0.293294,-0.315941,-0.301511,-0.291632,-0.286606,3.761649,-0.293294
1208.0,0.871252,0.825791,0.065727,-0.159055,-1.668130,1.080184,1.050000,-1.027606,0.721307,0.176938,...,-0.283221,-0.272888,-0.291632,-0.293294,-0.315941,-0.301511,-0.291632,-0.286606,-0.265841,-0.293294
1209.0,-0.397020,-1.073375,0.385426,0.208204,-0.886469,0.484223,-0.907064,0.545211,0.495767,0.070529,...,-0.283221,-0.272888,-0.291632,-0.293294,-0.315941,-0.301511,3.428979,-0.286606,-0.265841,-0.293294


In [52]:
y_train

,y
id,
0.0,74.0
1.0,51.0
2.0,70.0
3.0,52.0
4.0,85.0
...,...
1207.0,70.0
1208.0,86.0
1209.0,68.0


In [53]:
# selector = VarianceThreshold()
# x_train_tmp = selector.fit_transform(X,y)
#selector = SelectPercentile(mutual_info_regression,percentile=10)
from sklearn.feature_selection import RFECV
from sklearn.svm import SVR
from sklearn.feature_selection import SequentialFeatureSelector
#selector = SelectKBest(f_regression,k=100)
selector = SelectPercentile(f_regression,percentile=10)
#estimator = SVR(kernel="rbf")
#selector = SequentialFeatureSelector(estimator, n_features_to_select=100)
#selector = RFECV(estimator, step=20, cv=5,min_features_to_select = 100)
selector.fit(np.array(x_train), np.array(y_train).squeeze())
cols = selector.get_support(indices=True)
x_train.describe()
x_train = x_train.iloc[:,cols]
x_train.describe()
x_test = x_test.iloc[:,cols]
#dataset_selected_x


In [54]:
x_train.describe()

,x2,x15,x21,x23,x27,x29,x40,x69,x77,x87,...,x773,x774,x777,x788,x796,x801,x817,x819,x823,x824
count,969.000000,969.000000,969.000000,969.000000,969.000000,969.000000,969.000000,969.000000,969.000000,969.000000,...,969.000000,969.000000,969.000000,969.000000,969.000000,969.000000,969.000000,969.000000,969.000000,969.000000
mean,-0.026846,-0.031371,0.027210,0.020758,-0.005520,-0.006064,0.020985,-0.005543,0.008713,-0.008449,...,-0.004147,-0.032487,0.020746,-0.009164,-0.028209,0.024259,0.016066,-0.038697,-0.023970,-0.008959
std,0.969392,0.934263,0.953085,0.929277,0.958614,0.950476,0.946713,0.941111,0.973586,0.959143,...,0.962989,0.978649,0.946274,0.957594,0.937244,0.950534,0.979837,0.939467,0.949382,0.970368
min,-3.260324,-3.753661,-3.790056,-3.970377,-3.531529,-3.094758,-6.376107,-4.192139,-3.029386,-3.471346,...,-3.561294,-3.444544,-3.618466,-2.929883,-1.769302,-3.369238,-3.275529,-4.165737,-3.604930,-2.959420
25%,-0.625908,-0.481841,-0.622441,-0.448365,-0.599755,-0.531952,-0.479277,-0.486062,-0.580023,-0.533384,...,-0.545970,-0.625087,-0.492794,-0.640159,-0.650906,-0.538926,-0.528276,-0.520530,-0.531961,-0.633704
50%,-0.098680,-0.163774,0.030780,0.006993,0.019781,-0.090151,0.056461,-0.080896,0.013373,-0.059918,...,-0.098157,-0.002149,0.070586,-0.157854,-0.210679,0.032200,0.107782,-0.143959,-0.129176,-0.063026
75%,0.550273,0.448237,0.556733,0.595715,0.456922,0.575618,0.495668,0.520846,0.552545,0.479373,...,0.498189,0.486555,0.569029,0.485327,0.441775,0.639238,0.620386,0.478228,0.505974,0.571714
max,5.106786,5.823220,3.153101,3.964758,5.347886,4.317002,4.159437,4.713334,4.688262,5.303392,...,5.480392,4.713876,3.810136,4.631625,9.547696,3.553727,3.546799,4.437106,5.562047,4.476138


In [55]:
#x_train_tmp = pd.DataFrame()

In [56]:
# fsel = FeatureSelector(
#         problem_type='regression',
#         featsel_runs=10,
#         keep=None,
#         n_jobs=-1,
#         verbose=1,)

# new_X = fsel.fit_transform(x_train_tmp, y)
# autofeat_cols = new_X.columns.values
# autofeat_cols

In [57]:
# for j in range(832):
#         if j in accepted:
#           print (j)
#         else:
#           del dataset_selected_x[str(j)]
#dataset_selected_x = dataset_selected_x.iloc[:,cols]
# train_stats = dataset_selected_x.describe()
# train_stats = train_stats.transpose()
#dataset_selected_x

In [58]:
x_train.to_csv(preprocessed + 'X_train_preprocessed.csv',index_label="id")
x_test.to_csv(preprocessed + 'X_test_preprocessed.csv',index_label="id")
y_train.to_csv(preprocessed + 'y_train_preprocessed.csv',index_label="id")
y_test.to_csv(preprocessed + 'y_test_preprocessed.csv',index_label="id")

In [59]:
x_train.columns

Index(['x2', 'x15', 'x21', 'x23', 'x27', 'x29', 'x40', 'x69', 'x77', 'x87',
       ...
       'x773', 'x774', 'x777', 'x788', 'x796', 'x801', 'x817', 'x819', 'x823',
       'x824'],
      dtype='object', length=166)

In [60]:
x_train

,x2,x15,x21,x23,x27,x29,x40,x69,x77,x87,...,x773,x774,x777,x788,x796,x801,x817,x819,x823,x824
id,,,,,,,,,,,,,,,,,,,,,
0.0,-0.226678,-0.337851,-1.155732,0.599105,0.158027,-0.632436,0.495668,-0.494824,0.371962,0.436958,...,-0.311507,-0.265153,0.271874,-0.981772,-0.650906,-0.397318,0.871044,-0.181158,-0.412912,0.190751
1.0,1.587980,2.358662,1.345399,-0.121116,1.093737,0.658817,-1.435164,1.502663,0.376827,1.523190,...,1.711289,2.112223,0.399513,0.655927,0.497726,0.083890,-0.022647,1.789948,1.418251,-0.212470
2.0,-0.625908,-0.869980,-0.614856,0.005391,-0.406470,0.285827,0.356319,-0.368856,0.394291,0.298336,...,0.020761,-0.356562,0.087286,0.180971,-0.373038,0.639238,0.284144,-0.769021,0.360299,-0.437600
3.0,-2.929110,-3.152708,-0.441039,-1.832078,-3.384087,-0.505167,3.921911,-3.951556,-3.029386,-3.360842,...,-3.387495,-2.790336,-1.450183,-2.786642,-0.650906,-1.557200,-1.995884,-3.622290,-3.336300,-2.790125
4.0,-0.013582,-0.420911,1.085127,-0.267716,-0.174137,2.337714,-0.073531,0.093897,-0.829828,-0.111829,...,1.772192,-0.172693,-0.504449,-1.002283,1.632004,-0.648963,-0.549555,-0.002139,0.263169,0.016198
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1207.0,0.859382,0.678270,1.629350,0.994234,1.528276,1.362006,-1.770572,1.835961,0.455996,0.490772,...,4.135244,0.891160,0.657093,1.857465,-0.604746,0.979299,-0.362689,1.224911,2.146213,1.269247
1208.0,0.065727,0.327383,0.030918,-0.794895,0.463941,1.049338,-0.764547,0.773116,-0.918908,0.768645,...,0.644394,0.124948,-0.323168,-0.111129,0.286177,-0.389131,-0.871130,0.595737,0.715023,-0.120347
1209.0,0.385426,0.385349,2.354932,0.848814,-0.163240,-0.305340,-0.259389,0.253541,0.728109,-0.076369,...,1.545145,0.025123,0.070586,-0.560445,0.342699,0.887566,0.335091,0.024908,0.955451,0.104321
